# PyKale Tutorial: Drug-Target Interaction: DeepDTA

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/pykale/pykale/HEAD?filepath=examples%2Fbindingdb_deepdta%2Ftutorial.ipynb) 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pykale/pykale/blob/main/examples/bindingdb_deepdta/tutorial.ipynb)


## Setup

The first few blocks of code are necessary to set up the notebook execution environment and import the required modules, including PyKale.

This checks if the notebook is running on Google Colab and installs required packages.

In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    !pip install -q condacolab
    import condacolab
    condacolab.install()    

    !git clone https://github.com/pykale/pykale.git
    !git checkout bindingdb_deepdta_tutorial #delete before merging into main

    !conda env update -n base -f ~/pykale/environment.yml
    
    %cd examples/bindingdb_deepdta
else:
    print('Not running on CoLab')